# Introduction by Example

> [PyTorch geometric](https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html#learning-methods-on-graphs)의 튜토리얼을 구현한 jupyter notebook 입니다.

![](./images/pytorch_geometric.png)

## Learning Methods on Graphs

We will use a simple GCN layer and replicate the experiments on the Cora citation dataset. For a high-level explanation on GCN, have a look at its [blog post](http://tkipf.github.io/graph-convolutional-networks/).

### imports

In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

### data load

In [2]:
dataset = Planetoid(root='../data/Cora', name='Cora')

In [3]:
dataset

Cora()

### Model

Let’s implement a two-layer GCN:

- The constructor defines two `GCNConv` layers which get called in the forward pass of our network. Note that the non-linearity is not integrated in the `conv` calls and hence needs to be applied afterwards (something which is consistent accross all operators in PyTorch Geometric). Here, we chose to use ReLU as our intermediate non-linearity between and finally output a softmax distribution over the number of classes.

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)

### Train the Model

In [11]:
from tqdm.notebook import tqdm

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Net().to(device)
data = dataset[0].to(device)

In [10]:
optimizer = torch.optim.Adam(model.parameters(), 
                             lr=0.01, 
                             weight_decay=5e-4)

In [12]:
model.train()
for epoch in tqdm(range(200)):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

### Evaluate the Model

In [13]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8140
